In [ ]:
!pip install pandas_ta
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import pandas_ta as ta

In [ ]:
file_path = './data.xlsx'
data = pd.read_excel(file_path)

In [ ]:
data = data.sort_values(by='Date', ascending=True)

In [ ]:
# Adding indicators
data['RSI']=ta.rsi(data['Adj Close'], length=15)
data['EMAF']=ta.ema(data['Adj Close'], length=20)
data['EMAM']=ta.ema(data['Adj Close'], length=100)
data['EMAS']=ta.ema(data['Adj Close'], length=150)

data['Target'] = data['Adj Close']-data.Open
data['Target'] = data['Target'].shift(-1)

data['TargetClass'] = [1 if data.Target[i]>0 else 0 for i in range(len(data))]

data['TargetNextClose'] = data['Adj Close'].shift(-1)

data.dropna(inplace=True)
data.drop(['Volume', 'Date', 'Libellé'], axis=1, inplace=True)

In [ ]:
data.head(5)

,Adj Close,Open,High,Low,RSI,EMAF,EMAM,EMAS,Target,TargetClass,TargetNextClose
149,113.100,111.800,113.100,111.800,51.140820,85.234388,105.184035,135.295873,1.40,1,102.200
150,102.200,100.800,102.400,100.400,50.736786,86.850160,105.124945,134.857517,0.06,1,5.185
151,5.185,5.125,5.190,5.125,47.182086,79.072526,103.145936,133.140001,-0.10,0,43.150
152,43.150,43.250,43.250,43.000,48.689370,75.651333,101.957898,131.948080,0.01,1,27.360
153,27.360,27.350,27.585,27.260,48.077975,71.052158,100.480712,130.562807,-0.20,0,86.050


In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
data_set_scaled = sc.fit_transform(data)

In [ ]:
X = []

back = 30
print(data_set_scaled.shape[0])
for j in range(8):
    X.append([])
    for i in range(back, data_set_scaled.shape[0]):
        X[j].append(data_set_scaled[i-back:i, j])

X=np.moveaxis(X, [0], [2])

X, yi =np.array(X), np.array(data_set_scaled[back:,-1])
y=np.reshape(yi,(len(yi),1))

print(X)
print(X.shape)
print(y)
print(y.shape)

104261
[[[2.08163600e-01 2.01441441e-01 2.02651863e-01 ... 4.54206056e-01
   8.83099790e-01 1.00000000e+00]
  [1.88091208e-01 1.81621622e-01 1.83479671e-01 ... 4.63083082e-01
   8.82590474e-01 9.96691153e-01]
  [9.43770745e-03 9.23423423e-03 9.29940911e-03 ... 4.20352897e-01
   8.65532708e-01 9.83726802e-01]
  ...
  [3.97764841e-02 3.93873874e-02 3.91686095e-02 ... 3.00094932e-01
   7.09802815e-01 8.21675966e-01]
  [2.18218211e-02 2.16216216e-02 2.16000726e-02 ... 2.76406192e-01
   6.97314411e-01 8.11702047e-01]
  [8.65506829e-05 1.96396396e-04 1.95305509e-04 ... 2.48797765e-01
   6.83058764e-01 8.00680196e-01]]

 [[1.88091208e-01 1.81621622e-01 1.83479671e-01 ... 4.63083082e-01
   8.82590474e-01 9.96691153e-01]
  [9.43770745e-03 9.23423423e-03 9.29940911e-03 ... 4.20352897e-01
   8.65532708e-01 9.83726802e-01]
  [7.93504027e-02 7.79279279e-02 7.74950759e-02 ... 4.01556923e-01
   8.55292592e-01 9.74729815e-01]
  ...
  [2.18218211e-02 2.16216216e-02 2.16000726e-02 ... 2.76406192e-01
   

In [ ]:
len = int(len(X)*0.8)
X_train, X_test = X[:len], X[len:]
y_train, y_test = y[:len], y[len:]
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(y_train)

(83384, 30, 8)
(20847, 30, 8)
(83384, 1)
(20847, 1)
[[0.00728499]
 [0.05638685]
 [0.06776734]
 ...
 [0.08003455]
 [0.12443615]
 [0.0692342 ]]


In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import TimeDistributed

import tensorflow as tf
import keras
from keras import optimizers
from keras.callbacks import History
from keras.models import Model
from keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate
import numpy as np

np.random.seed(10)

lstm_input = Input(shape=(back, 8), name='lstm_input')
inputs = LSTM(150, name='first_layer')(lstm_input)
inputs = Dense(1, name='dense_layer')(inputs)
output = Activation('linear', name='output')(inputs)
model = Model(inputs=lstm_input, outputs=output)
adam = optimizers.Adam()
model.compile(optimizer=adam, loss='mse', metrics=[r2_score], run_eagerly=True)
model.fit(x=X_train, y=y_train, batch_size=64, epochs=30, shuffle=True, validation_split = 0.1)

Epoch 1/30
1173/1173 [==============================] - 409s 348ms/step - loss: 0.0162 - r2_score: -0.0322 - val_loss: 0.0165 - val_r2_score: -0.0518
Epoch 2/30
1173/1173 [==============================] - 362s 309ms/step - loss: 0.0160 - r2_score: -0.0134 - val_loss: 0.0162 - val_r2_score: -0.0220
Epoch 3/30
1173/1173 [==============================] - 387s 330ms/step - loss: 0.0160 - r2_score: -0.0120 - val_loss: 0.0165 - val_r2_score: -0.0303
Epoch 4/30
1173/1173 [==============================] - 376s 321ms/step - loss: 0.0159 - r2_score: -0.0098 - val_loss: 0.0162 - val_r2_score: -0.0207
Epoch 5/30
1173/1173 [==============================] - 391s 333ms/step - loss: 0.0159 - r2_score: -0.0101 - val_loss: 0.0163 - val_r2_score: -0.0281
Epoch 6/30
1173/1173 [==============================] - 377s 321ms/step - loss: 0.0159 - r2_score: -0.0093 - val_loss: 0.0162 - val_r2_score: -0.0200
Epoch 7/30
1173/1173 [==============================] - 379s 323ms/step - loss: 0.0159 - r2_score: -

In [ ]:
y_pred = model.predict(X_test)
print(r2_score(y_test, y_pred))
print(mean_squared_error(y_test, y_pred))

652/652 [==============================] - 84s 128ms/step
-6.175588435097801e-05
0.016688862097042133
